In [1]:
import Preprocessing as pp
import Global_Functions as gf
import datetime
from sklearn.metrics import mean_squared_error
from math import sqrt
from matplotlib import pyplot
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

from smt import surrogate_models as sm

import time

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [2]:
temp_data_folder = "../Data/Preped_Data/"
image_path = "../Images/"
image_subfolder = "Tries on Difference/"
image_folder = image_path + image_subfolder

gf.check_folder(temp_data_folder)
gf.check_folder(image_path)
gf.check_folder(image_folder)

Folder already exists.
Folder already exists.
Folder already exists.


In [3]:
df_1 = gf.open_CSV_file('experiment_1_short.csv', temp_data_folder)
df_4b = gf.open_CSV_file('experiment_4b_short.csv', temp_data_folder)
df_9 = gf.open_CSV_file('experiment_9_short.csv', temp_data_folder)

FileNotFoundError: [Errno 2] No such file or directory: '../Data/Preped_Data/experiment_4b_short.csv'

In [ ]:
timediff = 60
diff_1 = df_1.set_index('time').diff(timediff)
diff_4 = df_4b.set_index('time').diff(timediff)
diff_9 = df_9.set_index('time').diff(timediff)

In [ ]:
el_maxi = diff_4.loc[diff_4.idxmax()['el_power']]['el_power']
el_mini = diff_4.loc[diff_4.idxmin()['el_power']]['el_power']

In [ ]:
def scale3(train, test, val):
    scaler = MinMaxScaler(feature_range=(0,1))
    scaler = scaler.fit(train)
    
    train = train.reshape(train.shape[0], train.shape[1])
    train_scaled = scaler.transform(train)
    
    test = test.reshape(test.shape[0], test.shape[1])
    test_scaled = scaler.transform(test)
    
    val = val.reshape(val.shape[0], val.shape[1])
    val_scaled = scaler.transform(val)
    
    return scaler, train_scaled, test_scaled, val_scaled

In [ ]:
from keras.callbacks import EarlyStopping
es=EarlyStopping(monitor="val_loss",verbose=2,mode='min',patience=5)

In [ ]:
def fit_lstm(X_train, y_train, X_val, y_val, batch_size, nb_epoch, neurons):
    start = time.time()
    X, y = X_train, y_train
    X = X.reshape(X.shape[0], 1, X.shape[1])
    X_v = X_val.reshape(X_val.shape[0], 1, X_val.shape[1])
    model = Sequential()
    model.add(LSTM(units = 64, return_sequences = True, input_shape = (X_train.shape[1], 1)))
    model.add(Dropout(0.2))
                   
    model.add(LSTM(units = 64, return_sequences = True))
    model.add(Dropout(0.2))
    
    model.add(Dense(64, activation = "relu"))
    model.add(Dense(1, activation = "sigmoid"))
    model.compile(loss='mean_squared_error', optimizer='adam')
    for i in range(nb_epoch):
        model.fit(X, y, epochs=100, batch_size=batch_size, verbose=2, shuffle=False, validation_data=(X_v, y_val))
        model.reset_states()
    end = time.time()
    duration = end - start
    return model, duration

In [ ]:
y_train = df_4b['el_power']
X_train = df_4b['input_voltage']

X_val = df_9['input_voltage']
y_val = df_9['el_power']

y_test = df_1['el_power']
X_test = df_1['input_voltage']

test = [X_test, y_test]
train = [X_train, y_train]

In [ ]:
X_train = np.array(X_train[timediff:,])
X_test = np.array(X_test[timediff:,])
X_val = np.array(X_val[timediff:,])
y_train = np.array(y_train[timediff:,])
y_test = np.array(y_test[timediff:,])
y_val = np.array(y_val[timediff:,])

In [ ]:
dy_dx_train = np.array(diff_4['el_power'][timediff:,])
dy_dx_test = np.array(diff_1['el_power'][timediff:,])
dy_dx_val = np.array(diff_9['el_power'][timediff:,])

In [ ]:
scaler_X, X_train, X_test, X_val = scale3(X_train.reshape(-1,1), X_test.reshape(-1,1), X_val.reshape(-1,1))
scaler_y, y_train, y_test, y_val = scale3(y_train.reshape(-1,1), y_test.reshape(-1,1), y_val.reshape(-1,1))
scaler_dy_dx, dy_dx_train, dy_dx_test, dy_dx_val = scale3(dy_dx_train.reshape(-1,1), dy_dx_test.reshape(-1,1), dy_dx_val.reshape(-1,1))

In [ ]:
# X_train = X_train[timediff:].reshape(-1,1)
# #X_val = X_val[timediff:].reshape(-1,1)
# X_test = X_test[timediff:].reshape(-1,1)

In [ ]:
# dy_dx_train = dy_dx_train[timediff:].reshape(-1,1)
# dy_dx_val = dy_dx_val[timediff:].reshape(-1,1)
# dy_dx_test = dy_dx_test[timediff:].reshape(-1,1)

In [ ]:
# y_train = y_train[timediff:].reshape(-1,1)
# #y_val = y_val[timediff:].reshape(-1,1)
# y_test = y_test[timediff:].reshape(-1,1)

In [ ]:
lstm, dur = fit_lstm(X_train, y_train, X_val, y_val, 64, 1, 50)

In [ ]:
print('The training of the LSTM on difference dured %5.3f seconds.' %dur)

In [ ]:
sc = MinMaxScaler(feature_range= (el_mini, el_maxi))
sc.fit(y_test)

In [ ]:
X_test_reshaped = X_test[:, 0].reshape(len(X_test), 1, 1)
pred = lstm.predict(X_test_reshaped, batch_size=1)

In [ ]:
predictions = sc.transform(pred[:,0])
y_test_true = sc.transform(y_test)

In [ ]:
timestr = time.strftime("%Y-%m-%d_%H-%M")
pyplot.plot(y_test_true, label = 'true', color = gf.get_color('blue'))
pyplot.plot(predictions, label = 'LSTM', color = gf.get_color('orange'))

pyplot.legend()
pyplot.ylabel('Electrical power [W]')
pyplot.xlabel('time [s]')

pyplot.savefig(image_folder + timestr + '_ex4_LSTM_electric.svg')
pyplot.savefig(image_folder + timestr + '_ex4_LSTM_electric.png')
pyplot.show()

In [ ]:
ge = sm.genn.GENN()
ge.options['mini_batch_size'] = 128
ge.options['deep'] = 3
ge.options['wide'] = 10
ge.options['num_epochs'] = 20
sm.genn.load_smt_data(ge, X_train.astype(float), y_train.astype(float), dy_dx_train)
ge.train()
ge.plot_training_history()
ge.goodness_of_fit(X_val, y_val, dy_dx_val)

In [ ]:
y_pred = ge.predict_values(X_test.astype(float))

In [ ]:
plt.plot(y_test)
plt.plot(y_pred)